## Lab 9-1 : XOR을 위한 텐스플로우 딥넷트웍

In [1]:
import tensorflow as tf
import numpy as np

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### XOR data set

| A | B | X |
| - | - | - |
| 0 | 0 | 0 |
| 0 | 1 | 1 |
| 1 | 0 | 1 |
| 1 | 1 | 0 |

#### Boolean Expression
$$
X = A \oplus B
$$

#### Logic Diagram Symbol

<img src="./10.png" width="400" height="auto" alt="아직 안만듬">

[[이미지 출처]](https://mathphysics.tistory.com/579)

간단한 데이터셋이므로 따로 입력받지않고 **numpy array** 사용

In [2]:
x_data = np.array(
    [
        [0, 0],
        [0, 1],
        [1, 0],
        [1, 1],
    ],
    dtype=np.float32
)
y_data = np.array(
    [
        [0],
        [1],
        [1],
        [0],
    ],
    dtype=np.float32
)

<br/>

### Logistic Regression으로 해결하기
**0**인지 **1**인지 예측하는 **Logistic Regression**사용<br/>
$$X$$, $$Y$$에 맞게 **Weight**과 **Bias**를 결정해야한다.<br/>
**Weigth**을 정할때에는 **크기**가 **중요**하다.<br/>
입력값이 $$X_1$$, $$X_2$$로 2개고 출력값이 $$Y$$하나이기 때문에<br/>
**Weight**의 크기는 **[2, 1]**의 크기다.<br/>
**Bias**는 항상 **출력값의 갯수**와 같으므로 **[1]**이다.

In [3]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)
W = tf.Variable(tf.random_normal([2, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name="bias")

<br/>

### Hypothesis
**Sigmoid** 함수 사용<br/>
$$ Sigmoid = \frac{1}{1+e^{-x}}$$

**행렬 곱셉**을 사용한다.

In [4]:
hypothesis = tf.sigmoid(tf.matmul(X, W) + b)

<br/>

### Cost Function
**Logistic Regression**의 **Cost Function**은 아래와 같다.<br/>
$$
C(H(x), y) = 
- Y * log(H(x)
- (1 - Y) * log(1 - H(X))
$$

구해진 Cost를 가지고 **경사하강법**을 사용해 학습을 진행한다.

In [5]:
cost = -tf.reduce_mean(Y * tf.log(hypothesis)
                      + (1 - Y) * tf.log(1 - hypothesis))
train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

<br/>

### Training
위에 작성한 **Hypothesis**와 **Cost Function**을 사용해 학습을 진행<br/>
`cast`함수를 사용해 **Hypothesis**의 값이 **0.5**보다 클경우 **True**로<br/>
**0.5**보다 작을경우 **False**로 값을 바꾸어준다.<br/>
또한 예측값과 결과값을 비교해 `cast`함수를 사용한 결과의 평균을 구해

In [6]:
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(10001):
        _, cost_val, w_val = sess.run(
            [train, cost, W], feed_dict={X: x_data, Y: y_data}
        )

        if step % 1000 == 0:
            print(step, cost_val, w_val)
            
    h, c, a = sess.run(
        [hypothesis, predicted, accuracy], feed_dict={X: x_data, Y: y_data}
    )
    
    print("\nHypothesis: ", h, "\nCorrect: ", c, "\nAccuracy: ", a)

0 0.7531501 [[-0.7257929]
 [-0.9959548]]
1000 0.72031176 [[-0.56429285]
 [-0.7098626 ]]
2000 0.70586157 [[-0.3969825 ]
 [-0.47502416]]
3000 0.69901526 [[-0.27513653]
 [-0.3169331 ]]
4000 0.6958375 [[-0.18924381]
 [-0.21161896]]
5000 0.694377 [[-0.12953913]
 [-0.14151494]]
6000 0.69370866 [[-0.08838086]
 [-0.09479028]]
7000 0.69340336 [[-0.06015909]
 [-0.0635892 ]]
8000 0.6932641 [[-0.04087842]
 [-0.0427141 ]]
9000 0.69320047 [[-0.02774075]
 [-0.02872313]]
10000 0.6931715 [[-0.01880636]
 [-0.01933207]]

Hypothesis:  [[0.50565445]
 [0.5008217 ]
 [0.5009531 ]
 [0.49612015]] 
Correct:  [[1.]
 [1.]
 [1.]
 [0.]] 
Accuracy:  0.75
